In [19]:
import pandas as pd
from fuzzywuzzy import fuzz
import string
import pickle as pkl
import numpy as np

TRUE_CUTOFF = 3150
THRESHOLD = 85

replace_char = ["*", "&", "%", "/", "\\"]
strip_char = ["'", "-", ".", "!", ":", ";"]
num_char =  ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]
red_flag_char = [" AVE ", " ST ", " AV ", " BLVD ", " BLV ", " RD ", " DR "]
common_errors = {
    "0": "O",
    "1": "l",
    "5": "S",
    "8": "B"
}

# remake the dict
map_dict = {}

# use previous dict
# map_dict = pkl.load(open('trueheaders_match_dict.pkl', 'rb'))

def clean_header(h):
    red_flag = False
    for s in replace_char:
        h = h.replace(s, "")
    for s in strip_char:
        h = h.strip(s)
    cnt = 0
    hl = []
    for c in list(h):
        if c in common_errors: c = common_errors[c]
        if c in num_char: cnt += 1
        hl.append(c)
    h = ''.join(hl).upper()
    for rf in red_flag_char:
        if rf in h: red_flag = True
    if cnt > 3 or red_flag: h = ""
    return h
    
def assign_clean(D):
    return [clean_header(h) for h in D.true_headers]

def score(string1, string2):
    # Scores the fuzzy match between the ocr header and the true header
    return fuzz.ratio(string1,string2)

def match(headers, true_headers, map_dict):
    # Matches ocr headers to true headers
    for header in headers:
        if header not in map_dict:
            score_list = [(score(header, true_header), true_header) for true_header in true_headers]
            sorted_score = sorted(score_list, key = lambda tup: tup[0], reverse=True)
            score_tuple = sorted_score[0]
            if score_tuple[0] > 90:
                print("LOG: Matched! " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], score_tuple[1], "TRUE")
            else:
                print("LOG: Not matched, " + header + " and " + score_tuple[1] + " with score " + str(score_tuple[0]))
                map_dict[header] = (score_tuple[0], "no_header", "FALSE")
    return map_dict

raw = pd.read_csv("RankedHeaders.csv")[["count", "true_headers"]].dropna().assign(true_headers=assign_clean)

df = raw.drop_duplicates("true_headers")
df = df[df.true_headers.map(lambda h: (len(h) < 150) and (len(h) > 2) and (h is not ""))].reset_index(drop=True).sort_values("count", ascending=False)
true_headers = list(df[:TRUE_CUTOFF].true_headers)
unsure_headers = list(df[TRUE_CUTOFF:].true_headers)

In [7]:
map_dict = match(unsure_headers, true_headers, map_dict)

pkl.dump(map_dict, open('trueheaders_match_dict.pkl', 'wb'))

def assign_matched(D):
    matched = []
    for h in D.true_headers:
        if h in map_dict: 
            matched.append(map_dict[h][1])
        else: 
            print("Known: ", h, " not in map_dict")
            matched.append(h)
    return matched
def assign_score(D):
    scores = []
    for h in D.true_headers:
        if h in map_dict: 
            scores.append(map_dict[h][0])
        else:
            print("Known: ", h, " not in map_dict")
            scores.append(np.nan)
    return scores
def assign_bool(D):
    is_matched = []
    for h in D.true_headers:
        if h in map_dict:
            is_matched.append(map_dict[h][2])
        else:
            print("Known: ", h, " not in map_dict")
            is_matched.append("KNOWN")
    return is_matched
df = df.assign(matched = assign_matched, score = assign_score, is_matched = assign_bool).sort_values("count", ascending=False)

known = df.loc[df.matched != "no_header"].reset_index(drop=True).drop_duplicates().sort_values("count", ascending=False)
unknown = df.loc[df.matched == "no_header"].reset_index(drop=True).drop_duplicates("true_headers").sort_values("count", ascending=False)

def calculate_scores(df):
    l = len(df.true_headers)
    score_matrix = np.zeros((l,l))
    i = 0
    for h in df.true_headers:
        score_matrix[i, :] = df.true_headers.apply(score, args=(h,)).values
        print("Row number: {} of {}".format(i, l - 1))
        i += 1
    return score_matrix

def remove_repeat(df, scores):
    prelist = list(df.true_headers)
    i = 0
    for h in df.true_headers:
        j = 0
        for removal in scores[i, :]:
            if removal and j > i:
                try:
                    df = df[df["true_headers"] != prelist[j]]
                    print("LOG: " + prelist[j] + " deleted!")
                except KeyError:
                    print("LOG: " + prelist[j] + " Already deleted")
            j += 1
        i += 1
    return df

scores = calculate_scores(unknown)

removal_matrix = (scores > THRESHOLD) & (scores < 100)
internal_matched = remove_repeat(unknown, removal_matrix)
internal_matched = internal_matched.reset_index(drop=True)

LOG: Not matched, TOYS  GAMES-WHOLESALE and YAMS-WHOLESALE with score 80
LOG: Matched! HOSPITALS AND DISPENSARII'S  and HOSPITALS AND DISPENSARIES with score 93
LOG: Matched! INLUMBER-RETAIL and LUMBER-RETAIL with score 93
LOG: Matched! PICTURE FRAMES  PICTURES.- DEALERS and PICTURE FRAMES  PICTURES- DEALERS with score 99
LOG: Not matched, CEMENT, CEMENT PAINT  CEMENT PIPE and ENTERTAINMENT BUREAUS with score 52
LOG: Matched! HOTEL AND RESTAURANT SUNNLLES AND EQUIPMENT and HOTEL AND RESTAURANT SUPPLIES AND EQUIPMENT with score 93
LOG: Matched! HOSIERY-WHOLESALE  J OBBERS and HOSIERY-WHOLESALE  JOBBERS with score 98
LOG: Matched! PLORISTS-WHOLESALE and FLORISTS-WHOLESALE with score 94
LOG: Matched! REFRIGERATORS-SALOS AND SERVICE and REFRIGERATORS-SALES AND SERVICE with score 97
LOG: Matched! LAUNDRY SUPPLIES  EQUIPMENT -DEALERS and LAUNDRY SUPPLIES  EQUIPMENT DEALERS with score 99
LOG: Not matched, BROKERS-SGOTTO'N and BROKERS-GOLTTO with score 87
LOG: Matched! AUTOMOBILE ACCESSORIES  

LOG: Matched! BUTTER, CHEESE B; NGGS-RETAIL and BUTTER, CHEESE  NGS-RETAIL with score 95
LOG: Not matched, ICE DEALERS-PEDDLERS SUPPLIED and  JEWELERS, SUPPLIES with score 63
LOG: Not matched, LMOCIBTIIB and SOCIETIES with score 53
LOG: Matched! CONTRACTORS-PAVING  CONTRACTORS-PLASTERLNG and CONTRACTORS-PAVING  CONTRACTORS-PLASTERING with score 98
LOG: Not matched, PAINTERS-AUTO  CARRIAGE and PAINTERS-AUTOMOBILE  CARRIAGE with score 88
LOG: Matched! RADIATOR REPAIRERS- AUTOMOBILE and RADIATOR REPAIRERS-AUTOMOBILE with score 98
LOG: Matched! INSURANCE INVESTIGATIONS and INSURANCE INVESTIGATORS with score 94
LOG: Matched! MONUME NTAL WORK and MONUMENTAL WORK with score 97
LOG: Matched! LLABLLITY INSURANCE and LLABLLITV INSURANCE with score 95
LOG: Not matched, CUTLERY-RETAIL and HOSTERY-RETAIL with score 79
LOG: Not matched, TUBE  TUBING MIRS-METAL and STOVES  RANG IAS-RETAIL with score 61
LOG: Matched! PAINTERS-AUTOMOBLLE  CAR. RIAGE and PAINTERS-AUTOMOBILE  CARRIAGE with score 93
LOG: 

LOG: Matched! SHIN CHANDLERS and SHIP CHANDLERS with score 93
LOG: Matched! CONTRACTORS SUPPLIES AND EQUIPMENT-DEALERS AND REPAIRING and CONTRACTORS SUPPLIES AND EQUIPMENT- DEALERS AND REPAIRING with score 99
LOG: Not matched, ADI NSTERS and ADJ USTERS with score 80
LOG: Not matched, CALCULATING MACHINES and DUPLICATING MACHINES with score 80
LOG: Matched! ROOFING MATERIALS  ASPHALT SHINGLES and ROOTING MATERIALS  ASPHALT SHINGLES with score 97
LOG: Matched! CONTRACTORS -ROAD and CONTRACTORS-ROAD with score 97
LOG: Matched! GRANITE AND MARBLE WORKERS  and GRANITE AND MARBLE WORKERS with score 98
LOG: Matched! DRY GOODS- WHOLESALE and DRY GOODS-WHOLESALE with score 97
LOG: Matched! MUSICAL INSTRUMENT MIRS and MUSICAL INSTRUMENT MFRS with score 96
LOG: Matched! TYPEWFITING MACHINES-DEALERS and TYPEWRITING MACHINES-DEALERS with score 96
LOG: Not matched, TYPE FOUNDERS and IRON FOUNDERS  with score 67
LOG: Not matched, CINDER BLOCKS and CONCRETE BLOCK MFRS with score 63
LOG: Matched! OONTR

LOG: Matched! OIL BURNERS-SALI'B AND SERVICE and OIL BURNERS-SALN AND SERVICE with score 93
LOG: Not matched, CONTRACTORS- ACOUSTICAL and CONTRACTORS- ELECTRICAL with score 83
LOG: Not matched, EMBLEMS and EMBALMERS with score 75
LOG: Matched! PRODUCE DEALERS- WHOL and PRODUCE DEALERS-WHOI with score 93
LOG: Matched! LDTHOGRAPHERS and LITHOGRAPHERS with score 92
LOG: Matched! ARTIFICIAL EVES and ARTIFICIAL EYES with score 93
LOG: Matched! RADIATOR REPAIRERS-AUTOMOHILE and RADIATOR REPAIRERS-AUTOMOBILE with score 97
LOG: Matched! NOVELTIES-W'HOLESALE and NOVELTIES-WHOLESALE with score 97
LOG: Not matched, FOREIGN EXCHANGE AND and POREIGN EXCHANGE with score 83
LOG: Not matched, HEATING CONTRACTORS and EXCAVATING CONTRACTORS with score 88
LOG: Not matched, RAILROAD PASSENGER STATIONS and RADIO BROADCASTING STATIONS with score 74
LOG: Matched! CONTRACTORS and CONTRACTORS  with score 96
LOG: Not matched, ENGMEERS4-CWN and ENGINEERS-CONSULTI with score 65
LOG: Not matched, BOOKLET PRINTERS 

LOG: Not matched, GALCIUM CHLORIDE and VACUUM CLEANERS-DEALERS with score 51
LOG: Matched! PENCIL ' MFRS and PENCIL MFRS with score 92
LOG: Not matched, STAMP DEALERS AND PHILATELIC SUPPLIES and POSTAGE STAMPS AND PHILATELIC SUPPLIES with score 80
LOG: Not matched, ALUMINUM and ALUMINUM   with score 89
LOG: Matched! INVE STME NT COUNSE L and INVESTMENT COUNSEL with score 92
LOG: Not matched, HARDWARE DEALERS-WHOLESALE and HARDWARE DEALERS-WHOLESALE  JOBBERS with score 85
LOG: Not matched, TRANSPORTATLON-PRELUHT-ANTO- MOBILE and TRANSPORTATION-PREIGHT-AUTO- MOBILE with score 89
LOG: Not matched, CABARETS and BARBERS with score 67
LOG: Not matched, PARQUETRY FLOORING  PATENT ATTORNEYS and PATENT ATTORNEYS with score 62
LOG: Not matched, WORSTED GOODS MANUFACTURERS  WORSTED  MERINO GOODS MFRS and WORSTED  MERINO GOODS-MIRS with score 59
LOG: Matched! RECRUITLNG-ARMED SERVICES and RECRUITING-ARMED SERVICES with score 96
LOG: Not matched, UPHOLSTERING MATERIALS  SUP and BUILDING MATERIALS  

...

LOG: Not matched, DRAPERIES AND CURTAIN S and GROCERIES AND MEATS-RETAIL  with score 64
LOG: Not matched, STOCK BROKERS and BONDS  STOCKS-BROKERS with score 71
LOG: Not matched, RAILMADS  and RAILROADS  with score 84
LOG: Not matched, EMPLOYMENT AGENCIES-SECRE- TARIAL PLACEMENT BUREAUS and ENTERTAINMENT BUREAUS with score 56
LOG: Not matched, DERMATOLOGIST and DENTIST with score 60
LOG: Not matched, P-LLLOW SANITIZING and F'PILLOW SANITIZING with score 86
LOG: Not matched, INSURANEE-MARLNE and INSURANCE-MARINE with score 88
LOG: Matched! LOANS -AUTOMOBILE and LOANS-AUTOMOBILE with score 97
LOG: Not matched,  TITLE CO and TITLE INSURANCE with score 58
LOG: Not matched,  PLNS and  LOANS with score 73
LOG: Matched! FLOORS A and FLOORS  with score 93
LOG: Matched! TYPESE'ITING-MACHINE COM- POSITION and TYPESETTING-MACHINE COMPOSITION with score 92
LOG: Not matched, L'PRINTING PLATES and PRINTING PRESS MFRS with score 67
LOG: Not matched, CONCRETE -READY MIXED and ICE CREAM MIX with score 5

LOG: Matched! LEMBALMERS and EMBALMERS with score 95
LOG: Not matched, LCARRIAGE  WAGON MFRS  GALSEMENTS-STEEL and STORAGE-FURS AND GARMENTS with score 50
LOG: Not matched, SHLPPLNG TAG MFRS and SPRING MFRS with score 64
LOG: Not matched, HORSESHOEM  F and HORSESHOERS  HOSIERY-RETAIL with score 55
LOG: Not matched, RETRIGERATORS-ELEC,TRIC  GAS and REFRIGERATORS-ELECTRIC with score 84
LOG: Not matched, BINSO and SIGNS with score 60
LOG: Not matched, TUBE  TUBING MIRS-MEFTAL and STOVES  RANG IAS-RETAIL with score 60
LOG: Matched! TE LEGRAPH COMPANIES and TELEGRAPH COMPANIES with score 97
LOG: Matched! IINSURANCE COMPANIES and INSURANCE COMPANIES with score 97
LOG: Not matched, DOOR MANUFACTURERS- FOLDING and TOOL MANUFACTURERS with score 71
LOG: Not matched, BRLCK PACE  COMMO and BRICK, LIME  CEMENT with score 56
LOG: Not matched, GUTTERS AND CONDUCTOR PIPE -GALVANI ZED STEEL AND COPPER and GUTTERS  CONDUCTOR PIPE  with score 60
LOG: Not matched, CAULKJNG COMPOUNDS AND GUNS and CLEANING 

LOG: Not matched, CLOTHING MFRS-CHILDRE and CLOTHING MFRS-WOMEN'S with score 71
LOG: Not matched, CANDLCS and CHAINS with score 62
LOG: Not matched, CLEANING COMPOUNDS-MANU- FACTURERS and CLEANING COMPOUNDS-MFRS with score 81
LOG: Not matched, WATERPROOFING COMPOSITION and WATERPROOFING with score 68
LOG: Matched! JEWELERS--SPECIAL ORDER WORK and JEWELERS-SPECIAL ORDER WORK with score 98
LOG: Not matched, ANNUITIEB and ANTIQUES with score 59
LOG: Not matched, FIRE PROTECTION SPRINKLER SYSTEMS and FIRE PROTECTION SERVICE with score 71
LOG: Not matched, TBOILER INSTALLATIONS O'CONNOR JOHN F PLUMBING  HEATIM and BOILER INSTALLATIONS AND REPAIRS with score 60
LOG: Not matched, OLASS PINS  RINGS and ASSOC I AT IONS with score 63
LOG: Matched! MOVERS-HOUSIEHOLD GOODS and MOVERS-HOUSEHOLD GOODS with score 98
LOG: Not matched, WOMEN'S SPECIALTY SHOPS and WIRE SPECIALTIES with score 62
LOG: Not matched, L' PRINTERS-OFFSET and  PHOTO-OFFSET with score 65
LOG: Not matched, NET AND SEINE MAKERS an

LOG: Not matched, SPRAY PAINTING and PAINTING with score 73
LOG: Matched! ELECTRLC MOTOR REPAIRS and ELECTRIC MOTOR REPAIRS with score 95
LOG: Not matched,  GLOCKS-D EALECRS-WHOLESALE and GOAL DEALERS-WHOLESALE with score 78
LOG: Not matched, HEATINQ  PLUMBING and PLUMBING with score 64
LOG: Not matched, WULLEYS and BOWLING ALLEYS with score 57
LOG: Not matched, CLOTHING-BOYS'-RETALL and CLOTHING-RETAIL with score 78
LOG: Not matched, PAR TS-MANUFAC TURERS and PARTS MANUFACTURERS with score 90
LOG: Not matched, CLOTHES DRYERS-ELECTRIC and CONTRACTORS-ELECTRICAL with score 67
LOG: Not matched, SAIE DEALERS AND EXPERTS and TIRE DEALERS AND REPAIRERS with score 80
LOG: Not matched, BUJLDING CONTRACTORS and PAINTING CONTRACTORS with score 75
LOG: Not matched, CEMENT COLORS and INVESTMENT COUNSELORS with score 71
LOG: Not matched, WOOLEN GOODS- MANUFACTURERS and WIRE GOODS MANUFACTURERS with score 86
LOG: Not matched, WATER W ORKS SYSTEMS and TEMPERATURE CONTROL SYSTEMS with score 60
LOG: M

LOG: Not matched, STAMNINGS-METAL and STATIONERS-RETAIL with score 69
LOG: Not matched, AUTOMATIC TYPEWRITING and AUTOMOBILE PARKING with score 62
LOG: Not matched, GESIDE-NTIAL PAINTERS and SIGN PAINTERS MRS with score 63
LOG: Not matched, ELECTRIC MOTORS AND  GENERATORS-MFRS and ELECTRIC MOTORS  GENERATORS with score 86
LOG: Not matched, E LASTIC STOCKINGS and STOCKINGS with score 67
LOG: Not matched,  BUS AND TRUCK BATTERIES- HEAVY DUTY and C URTAINS AND DRAPERIES with score 51
LOG: Not matched, LINENS-RETAIL  LINOTYPE  MONOTYPE GOM- POSITIMI and INSTALLMENT GOODS with score 47
LOG: Not matched, LWZ'RUTTERS AND CONDUCTOR PIPE-GALVANIZED STEEL, COPPER AND ALUMINUM and GUTTERS  CONDUCTOR PIPE  with score 50
LOG: Not matched, POSTAUE STAMP COLLECTORS  and PATENT SOLICITORS with score 57
LOG: Not matched, COMPUTER SCHOOLS and GOLF SCHOOLS with score 64
LOG: Matched! CLCTHING-WOMENS-RETAIL and CLOTHING-WOMEN'S-RETAIL with score 93
LOG: Not matched, FENCE ERECTORS GENERAL FENCE SUPPLY COM